In [1]:
import boa
from utils import deploy_infra, deploy_pool, get_balances
from IPython.display import display, HTML

## **Table of Content**

1. [Infrastrucutre and Pool Setup](#infrastrucutre-and-pool-setup)
2. [Calculate LP Tokens When using `remove_liquidity`](#2-calculate-lp-token-amount)
3. [Remove Liquidity](#2-remove-liquidity-balanced)
4. [Calculate tokens using `remove_liquidity_one_coin`](#4-calculate-withdraw-one-coin)
5. [Remove Liquidity Single-Sided](#3-remove-liquidity-single-sided)

### **1. Infrastrucutre and Pool Setup**

Before we can exchange tokens, we need to deploy the twocrypto-ng infrastructure and deploy a pool. Afterwards, we grant approval to the pool and add some initial liquidity.

In [2]:
# deploying infra for twocrypto-ng pools
infra = deploy_infra()
factory = infra["factory"]
trader = infra["trader"]
coin_a = infra["coin_a"]
coin_b = infra["coin_b"]
fee_receiver = infra["fee_receiver"]

# deploy pool
pool = deploy_pool(factory, coin_a, coin_b)
pool = infra["impl"].at(pool)
pool

/Users/mo/Documents/CurveDocs/boa/lib/python3.12/site-packages/vyper/ast/folding.py:153: UserWarning: ZERO_ADDRESS is deprecated. Please use `empty(address)` instead.
  warnings.warn(f"{name} is deprecated. Please use `{replacement}` instead.")
/Users/mo/Documents/CurveDocs/boa/lib/python3.12/site-packages/vyper/semantics/types/subscriptable.py:115: UserWarning: Use of large arrays can be unsafe!
  warnings.warn("Use of large arrays can be unsafe!")


<../contracts/main/CurveTwocryptoOptimized.vy at 0x5Bac22268eecccDF2fa09D03eC0D0fDbA2848ba2, compiled with vyper-0.3.10+9136169>
<storage: cached_price_scale=1000000000000000000, cached_price_oracle=1000000000000000000, cached_xcp_oracle=0, last_prices=1000000000000000000, last_timestamp=580937928559420126894148428052308409693426034781, last_xcp=0, xcp_ma_time=62324, initial_A_gamma=136112946768375385385349842972852284582400000, initial_A_gamma_time=0, future_A_gamma=136112946768375385385349842972852284582400000, future_A_gamma_time=0, balances=[0, 0], D=0, xcp_profit=0, xcp_profit_a=1000000000000000000, virtual_price=0, packed_rebalancing_params=680564733841876929619973849625130958848000000000865, packed_fee_params=8847341539944400050877843276543133320576000000, last_admin_fee_claim_timestamp=0, admin_lp_virtual_balance=0, balanceOf={}, allowance={}, totalSupply=0, nonces={}>

In [3]:
# grant max approval and add some initial liquidity

with boa.env.prank(trader):
    for coin in [coin_a, coin_b]:
        coin.approve(pool, 2**256-1)

with boa.env.prank(trader):
    pool.add_liquidity([10**21, 10**21], 0)

print("Pool Balance CoinA:", pool.balances(0) / 1e18)
print("Pool Balance CoinB:", pool.balances(1) / 1e18)

Pool Balance CoinA: 1000.0
Pool Balance CoinB: 1000.0


### **2. Calculate LP Token Amount**

- #### `calc_token_amount(amounts: uint256[N_COINS], deposit: bool) -> uint256:`

    Function to calculate the LP tokens to be minted or burned for depositing or removing `amounts` of coins. This method takes fees into consideration.

    Returns: Amount of LP tokens deposited or withdrawn (`uint256`).

    | Input      | Type               | Description                                     |
    | ---------- | ------------------ | ----------------------------------------------- |
    | `amounts`  | `uint256[N_COINS]` | Amounts of tokens being deposited or withdrawn. |
    | `deposit`  | `bool`             | `true` for deposit, `false` for withdrawal.     |

In [4]:
pool.calc_token_amount([10**20, 10**20], False)

99998999999999999999

### **3. Remove Liquidity**

- #### `remove_liquidity(_amount: uint256, min_amounts: uint256[N_COINS], receiver: address = msg.sender) -> uint256[N_COINS]:`

    Function to remove liquidity from the pool and burn `_amount` of LP tokens. When removing liquidity with this function, no fees are charged as the coins are **withdrawn in balanced proportions**. This function also updates the `xcp_oracle` since liquidity was removed.

    Returns: Withdrawn balances (`uint256[N_COINS]`).

    Emits: `RemoveLiquidity`

    | Input         | Type                | Description                                      |
    | ------------- | ------------------- | ------------------------------------------------ |
    | `_amount`     | `uint256`           | Amount of LP tokens to burn.                     |
    | `min_amounts` | `uint256[N_COINS]`  | Minimum amounts of tokens to withdraw.           |
    | `receiver`    | `address`           | Receiver of the coins; defaults to `msg.sender`. |

In [5]:
display(HTML(get_balances(trader, pool, coin_a, coin_b).to_html(index=False)))

with boa.env.prank(trader):
    pool.remove_liquidity(10**20, [0, 0], trader)

display(HTML(get_balances(trader, pool, coin_a, coin_b).to_html(index=False)))

Asset,Balance
Trader: Balance A,99000.0
Trader: Balance B,99000.0
Trader: LP Tokens,1000.0
Pool: Balance A,1000.0
Pool: Balance B,1000.0


Asset,Balance
Trader: Balance A,99100.0
Trader: Balance B,99100.0
Trader: LP Tokens,900.0
Pool: Balance A,900.0
Pool: Balance B,900.0


As seen, `trader` withdrew 100 LP tokens in a balanced proportion and received 100 coinA and 100 coinB.

### **4. Calculate Withdraw One Coin**

- #### `calc_withdraw_one_coin(token_amount: uint256, i: uint256) -> uint256:`

    Function to calculate the amount of output token `i` when burning `token_amount` of LP tokens. This method takes fees into consideration.

    Returns: Amount of tokens received (`uint256`).

    | Input         | Type      | Description                              |
    | ------------- | --------- | ---------------------------------------- |
    | `token_amount`| `uint256` | Amount of LP tokens burned.              |
    | `i`           | `uint256` | Index of the coin to withdraw.           |

In [6]:
pool.calc_withdraw_one_coin(10**20, 0) / 1e18

189.65554321474175

### **5. Remove Liquidity Single-Sided** 

- #### `remove_liquidity_one_coin(token_amount: uint256, i: uint256, min_amount: uint256, receiver: address = msg.sender) -> uint256:`

    Function to burn `token_amount` LP tokens and withdraw liquidity in a single token `i`.

    Returns: Amount of coins withdrawn (`uint256`).

    Emits: `RemoveLiquidityOne`

    | Input          | Type      | Description                                      |
    | -------------- | --------- | ------------------------------------------------ |
    | `token_amount` | `uint256` | Amount of LP tokens to burn.                     |
    | `i`            | `uint256` | Index of the token to withdraw.                  |
    | `min_amount`   | `uint256` | Minimum amount of token to withdraw.             |
    | `receiver`     | `address` | Receiver of the coins; defaults to `msg.sender`. |

In [7]:
display(HTML(get_balances(trader, pool, coin_a, coin_b).to_html(index=False)))

with boa.env.prank(trader):
    pool.remove_liquidity_one_coin(10**20, 0, 0, trader)

display(HTML(get_balances(trader, pool, coin_a, coin_b).to_html(index=False)))

Asset,Balance
Trader: Balance A,99100.0
Trader: Balance B,99100.0
Trader: LP Tokens,900.0
Pool: Balance A,900.0
Pool: Balance B,900.0


Asset,Balance
Trader: Balance A,99289.655543
Trader: Balance B,99100.000000
Trader: LP Tokens,800.000000
Pool: Balance A,710.344457
Pool: Balance B,900.000000
